In [1]:
import os
import numpy as np
import pandas as pd
import re
from tqdm import tqdm

from src.Utils import tools

In [2]:
seasons_dicts=tools.create_seasons_dict(start_year=2007,
                                          end_year=2022)

In [3]:
seasons_dicts

[{'season': '2007-08', 'years': [2007, 2008]},
 {'season': '2008-09', 'years': [2008, 2009]},
 {'season': '2009-10', 'years': [2009, 2010]},
 {'season': '2010-11', 'years': [2010, 2011]},
 {'season': '2011-12', 'years': [2011, 2012]},
 {'season': '2012-13', 'years': [2012, 2013]},
 {'season': '2013-14', 'years': [2013, 2014]},
 {'season': '2014-15', 'years': [2014, 2015]},
 {'season': '2015-16', 'years': [2015, 2016]},
 {'season': '2016-17', 'years': [2016, 2017]},
 {'season': '2017-18', 'years': [2017, 2018]},
 {'season': '2018-19', 'years': [2018, 2019]},
 {'season': '2019-20', 'years': [2019, 2020]},
 {'season': '2020-21', 'years': [2020, 2021]},
 {'season': '2021-22', 'years': [2021, 2022]}]

In [4]:
def clean_date_field(date,first_season_year,second_season_year):
    
    date_array = date.split('-')

    month = date_array[2][:2]
    day = date_array[2][2:]

    if month[0] == '0':
        month = int(month[1:])

    if int(month) >= 10:
        year=first_season_year
    else:
        year=second_season_year
        
    return f'{year}-{month}-{day}'

In [5]:
results_dicts=[]
for season_dict in tqdm(seasons_dicts):
    
    season=season_dict['season']
    years=season_dict['years']
    first_season_year=years[0]
    second_season_year=years[1]

    #Retrieve odds data, preprocess and convert to dictionary
    odds_dataframe_file = pd.read_excel( f'Odds-Data/Odds-Data-Clean/{season}.xlsx')
    odds_dataframe_file=odds_dataframe_file.drop(columns='Unnamed: 0',axis=1,errors='ignore')
    odds_dataframe_file['Date']=odds_dataframe_file['Date'].apply(clean_date_field,args=(first_season_year,second_season_year))
    odds_dicts=pd.DataFrame(odds_dataframe_file).to_dict(orient="records")

    for odds_dict in tqdm(odds_dicts):

        home_team_name=odds_dict['Home']
        away_team_name=odds_dict['Away']
        game_date=odds_dict['Date']
        
        #Parse date
        date_array = game_date.split('-')
        year = date_array[0]
        month = date_array[1]
        day = date_array[2]
        
        if month[0] == '0':
            month = month[1:]
        if day[0] == '0':
            day = day[1:]

        #Get stats data
        try:
            team_stats_dataframe_file = pd.read_excel(f'Team-Data/{season}/{year}-{month}-{day}.xlsx')
        except FileNotFoundError:
            print(f'Stats not found for {year}-{month}-{day}')
            pass
        
        team_stats_dataframe_file=team_stats_dataframe_file.drop(columns=['Unnamed: 0','CFPARAMS','TEAM_ID','CFID'],axis=1,errors='ignore')
        team_stats_dicts=pd.DataFrame(team_stats_dataframe_file).to_dict(orient="records")
        
        #Replace old team names
        for dic in team_stats_dicts:
            if dic['TEAM_NAME'] == 'New Orleans Hornets':
                dic['TEAM_NAME'] = 'New Orleans Pelicans'

        #Get stats
        home_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==home_team_name and dic['Date']==game_date]
        away_team_stats_list=[dic for dic in team_stats_dicts if dic['TEAM_NAME']==away_team_name and dic['Date']==game_date]
        
        #Check if there is team stats data
        if len(home_team_stats_list)>0 and len(away_team_stats_list)>0:
            
            #Filter for stats
            home_team_stats_dict={k+'_HOME': v for k, v in home_team_stats_list[0].items() if k != 'Date' and k != 'TEAM_NAME'}
            away_team_stats_dict={k+'_AWAY': v for k, v in away_team_stats_list[0].items() if k != 'Date' and k != 'TEAM_NAME'}

            #Combine stats and odds
            team_result_dict = {**odds_dict , **home_team_stats_dict, **away_team_stats_dict}

            #Compile results
            results_dicts.append(team_result_dict)

 49%|██████████████████████████████████████▋                                        | 644/1316 [00:15<00:16, 41.37it/s]


 99%|█████████████████████████████████████████████████████████████████████████████▎| 1305/1316 [00:31<00:00, 41.57it/s]


 49%|██████████████████████████████████████▋                                        | 645/1315 [00:15<00:15, 42.47it/s]


 98%|████████████████████████████████████████████████████████████████████████████▊ | 1294/1315 [00:31<00:00, 39.70it/s]


 49%|██████████████████████████████████████▎                                        | 637/1312 [00:15<00:16, 41.22it/s]


...

  0%|                                                                                         | 0/1171 [00:00<?, ?it/s]

Stats not found for 2019-10-2
Stats not found for 2019-10-4
Stats not found for 2019-10-6
Stats not found for 2019-10-9
Stats not found for 2019-10-11



 56%|████████████████████████████████████████████▎                                  | 656/1171 [00:16<00:12, 42.24it/s]


 23%|██████████████████▋                                                             | 142/608 [00:03<00:11, 39.79it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [07:28<00:00, 29.88s/it]


In [6]:
df=pd.DataFrame(results_dicts)

In [7]:
df['Home-Team-Win']=df['Win_Margin'].apply(lambda x: 'W' if x >0 else 'L')

In [8]:
df['OU'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]', value=r'')

In [9]:
df['Points']=df['Points'].astype(int)
df['OU']=df['OU'].astype(float)

In [10]:
df['OU-Cover'] = np.where(df['Points']>df['OU'], 'Over', np.where(df['Points']<df['OU'], 'Under', 'Even'))

In [11]:
df.to_csv('Datasets/Full-Data-Set-UnderOver-2021-22.csv')